# Dean Property Consulting 2024

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [46]:
# load dataset
df = pd.read_csv('../data/df_lownull.csv', parse_dates=True)
df = df.drop(columns=['Unnamed: 0'])
# clean column names (lowercase and remove spaces)
df.columns = [col.lower().replace(' ', '').replace('/', '_') for col in df.columns]
df['annual$'] = df['annual$'].str.replace('$','').str.replace(',','').astype(float)
df['date'] = df['date'].astype('datetime64[ns]')
df['sale_ask'] = df['sale_ask'].astype('float')
df.head()

,date,sale_ask,units,floors,sf,annual$
0,2010-01-03,770000.0,1,2.5,"2,722",36033.0
1,2010-01-06,1397478.0,2,3,"3,780",63202.0
2,2010-01-10,505000.0,1,1,800,30089.0
3,2010-01-11,450000.0,1,1,650,31473.0
4,2010-01-11,1530000.0,3,3,"3,200",69400.0


In [47]:
for col in df.columns:
    print(set(df[col].apply(type)))

{<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{<class 'float'>}
{<class 'float'>, <class 'str'>}
{<class 'float'>, <class 'str'>}
{<class 'float'>, <class 'str'>}
{<class 'float'>}


In [49]:
# see only the rows with string-type
units_string = df[df['units'].apply(lambda x: isinstance(x, str))]
floors_string = df[df['floors'].apply(lambda x: isinstance(x, str))]
sf_string = df[df['sf'].apply(lambda x: isinstance(x, str))]
units_string

,date,sale_ask,units,floors,sf,annual$
0,2010-01-03,770000.0,1,2.5,"2,722",36033.0
1,2010-01-06,1397478.0,2,3,"3,780",63202.0
2,2010-01-10,505000.0,1,1,800,30089.0
3,2010-01-11,450000.0,1,1,650,31473.0
4,2010-01-11,1530000.0,3,3,"3,200",69400.0
...,...,...,...,...,...,...
40679,2023-12-14,470000.0,2,2,"1,966",24434.0
40680,2023-12-14,2220000.0,1,2,"1,872",115411.0
40681,2023-12-14,690000.0,1,2,986,35871.0
40682,2023-12-14,850000.0,1,3,"1,660",44189.0


In [50]:
df['units'] = df['units'].astype(float)

ValueError: could not convert string to float: '2 bdr, 1 bath, 2 weeks, common $445, tax $1968, Corcoran'

In [34]:
for col in df.columns:
    print('column:', col)
    print('unique values:', df[col].unique())
    print('minimum:', df[col].min())
    print('maximum:', df[col].max())
    print('\n')

column: date
unique values: ['2010-01-03' '2010-01-06' '2010-01-10' ... '2023-12-12' '2023-12-13'
 '2023-12-14']
minimum: 2010-01-03
maximum: 2023-12-14


column: sale_ask
unique values: [ 770000. 1397478.  505000. ... 2924000. 2035482. 1366000.]
minimum: 0.0
maximum: 102000000.0


column: units
unique values: ['1' '2' '3' '8' '4' '5' '7' '6' '10' '35' '53' '12' '0' '9' '19' '24'
 '40' '21' '36' '26' '16' '20' nan '14' '60' '23' '33' '15' '37' '13' '30'
 '17' '11' '27' '18'
 '2 bdr, 1 bath, 2 weeks, common $445, tax $1968, Corcoran'
 '1 bdr, 1 bath, 17 weeks, 500 SF, $480 common, 57% tax Corcoran' '192'
 '22' '25' '31' '80' '41' '32' '68' '134' '34' '42' '54' '70' '49' '28'
 '48' '43' '39' '38' '29' '62' '46' '56' '44' '52' '47' '51' '86' '167']


TypeError: '<=' not supported between instances of 'str' and 'float'